This notebook applies the same cleaning pipeline you used on training data to the **test** dataset, ensuring consistent features for modeling.



In [3]:
import pandas as pd
import numpy as np
import os

In [4]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
data_path = os.path.join(parent_dir, 'data')
out_path = os.path.join(parent_dir, 'outputs')     # For the output

In [9]:
# Read raw test
test = pd.read_csv(os.path.join(data_path, 'test.csv'))
train = pd.read_csv(os.path.join(data_path, 'train.csv'))

In [10]:
# Mark test and combine
test['status_group'] = np.nan
df = pd.concat([train, test], sort=False).reset_index(drop=True)

In [11]:
##  Date features
df['date_recorded'] = pd.to_datetime(df['date_recorded'])
df['recorded_year'] = df['date_recorded'].dt.year

In [13]:
# Replace invalid numeric entries
# Construction year: 0 is invalid
df['construction_year'] = df['construction_year'].replace(0, np.nan)


In [14]:
# GPS height: <= 0 invalid
df['gps_height'] = df['gps_height'].apply(lambda x: np.nan if x <= 0 else x)


In [15]:
# Longitude: 0 invalid
df['longitude'] = df['longitude'].replace(0, np.nan)

In [16]:
# Ensure lat/lon numeric
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')

In [17]:
##  Impute `construction_year`

df['construction_year'] = df.groupby(['region', 'installer'])['construction_year'].transform(
    lambda x: x.fillna(x.median())
)
df['construction_year'] = df.groupby('region')['construction_year'].transform(
    lambda x: x.fillna(x.median())
)
# Fallback to recorded_year - 13 (median lag)
df['construction_year'] = df['construction_year'].fillna(df['recorded_year'] - 13)

D:\script\Datacircle\water_pumps\.venv\lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
D:\script\Datacircle\water_pumps\.venv\lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
D:\script\Datacircle\water_pumps\.venv\lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
D:\script\Datacircle\water_pumps\.venv\lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
D:\script\Datacircle\water_pumps\.venv\lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
D:\script\Datacircle\water_pumps\.venv\lib\site-packages\num

In [18]:
##  Impute `gps_height`
df['gps_height'] = df.groupby('lga')['gps_height'].transform(lambda x: x.fillna(x.median()))
df['gps_height'] = df.groupby('region')['gps_height'].transform(lambda x: x.fillna(x.median()))

D:\script\Datacircle\water_pumps\.venv\lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
D:\script\Datacircle\water_pumps\.venv\lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
D:\script\Datacircle\water_pumps\.venv\lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
D:\script\Datacircle\water_pumps\.venv\lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
D:\script\Datacircle\water_pumps\.venv\lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
D:\script\Datacircle\water_pumps\.venv\lib\site-packages\num

In [19]:
## Clean latitude and longitude ranges

df['latitude'] = df['latitude'].apply(lambda x: x if (-15 <= x <= 0) else np.nan)
df['longitude'] = df['longitude'].apply(lambda x: x if (29 <= x <= 42) else np.nan)

In [21]:
## Impute `population`
# Treat 0 as missing
df['population'] = df['population'].replace(0, np.nan)
# Median by lga, then region, then overall median
df['population'] = df.groupby('lga')['population'].transform(lambda x: x.fillna(x.median()))
df['population'] = df.groupby('region')['population'].transform(lambda x: x.fillna(x.median()))
df['population'] = df['population'].fillna(df['population'].median())

D:\script\Datacircle\water_pumps\.venv\lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
D:\script\Datacircle\water_pumps\.venv\lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
D:\script\Datacircle\water_pumps\.venv\lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
D:\script\Datacircle\water_pumps\.venv\lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
D:\script\Datacircle\water_pumps\.venv\lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
D:\script\Datacircle\water_pumps\.venv\lib\site-packages\num

In [22]:
##  Fill categorical missing values
cat_cols = df.select_dtypes(include='object').columns
df[cat_cols] = df[cat_cols].fillna('Unknown')

In [25]:
##  Finalize and save
# Extract cleaned test rows
test_clean = df[df['status_group'].isna()].copy()
# Drop helper columns
test_clean = test_clean.drop(columns=['status_group'])

# Save cleaned test set
out_file = os.path.join(data_path, "test_cleaned.csv")
test_clean.to_csv(out_file)
print("Cleaned test data saved to outputs/test_cleaned.csv")


Cleaned test data saved to outputs/test_cleaned.csv
